# Ogum ML Derivatives & Arrhenius Demo

This notebook showcases the new preprocessing pipeline for Ogum ML Lite.
We generate a synthetic dataset, compute derivatives, evaluate Arrhenius
activation energies and assemble the stage-aware feature store.

In [ ]:
import numpy as np
import pandas as pd
from ogum_lite.io_mapping import ColumnMap, apply_mapping
from ogum_lite.preprocess import derive_all
from ogum_lite.arrhenius import arrhenius_feature_table
from ogum_lite.features import build_feature_store

# Synthetic raw spreadsheet mimicking Ogum 6.4 aliases
time = np.linspace(0, 1800, 181)
raw_df = pd.DataFrame(
    {
        'Sample': ['Demo'] * time.size,
        'tempo_min': time / 60,
        'temp_K': 873.15 + 0.12 * time,
        'densidade_relativa': 0.4 + 0.6 * (1 - np.exp(-time / 600)),
        'composition': 'Fe-2Cu',
        'technique': 'Flash',
    }
)

mapping = ColumnMap(
    sample_id='Sample',
    time_col='tempo_min',
    temp_col='temp_K',
    y_col='densidade_relativa',
    composition='composition',
    technique='technique',
    time_unit='min',
    temp_unit='K',
)
canonical = apply_mapping(raw_df, mapping)
canonical.head()

In [ ]:
derived = derive_all(canonical, smooth='savgol', window=9, poly=3)
arrhenius = arrhenius_feature_table(derived)
feature_store = build_feature_store(derived, theta_ea_kj=[200.0, 250.0])

display(arrhenius)
display(feature_store.head())